In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [1]:
import requests
import pandas as pd
import numpy as np
import math
import json
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter
from bokeh.models.widgets import Tabs, Panel
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from IPython.display import Markdown as md
import os

# Initialize Bokeh for charts
output_notebook()

# Pull latest data
url = "https://covidtracking.com/api/states"
r = requests.get(url)

# Convert data to json
j = r.json()

# Load data into dataframe
sars = pd.DataFrame(j)

# COLUMN REFERENCE
# stats: state, positive, negative, pending, hospitalized, death, lastUpdateEt, totalTestResults, grade, score

# Drop columns not currently being used
drop_columns = ["positiveScore",
                "negativeScore",
                "negativeRegularScore",
                "commercialScore",
                "total",
                "checkTimeEt",
                "fips",
                "dateModified",
                "dateChecked",
                "notes",
                "hash",
               ]
sars.drop(columns=drop_columns, inplace=True)

# Drop states
drop_states = ["PR",
               "AS",
               "GU",
               "MP",
               "VI",
              ]
for state in drop_states:
    sars.drop(sars[sars.state == state].index, inplace=True)

# Fill any missing data with zeroes
sars['pending'].fillna(0, inplace=True)
sars['hospitalized'].fillna(0, inplace=True)
sars['death'].fillna(0, inplace=True)
sars['negative'].fillna(0, inplace=True)
sars['score'].fillna(0, inplace=True)


# Load State Population file
with open('state_populations.json') as data:
    state_pop = pd.read_json(data)

# Merge state population data with covid data into new covid dataframe
covid = pd.merge(left=sars, right=state_pop[['state_code','state_name','population']], how='left', left_on='state', right_on='state_code')

# Create new columns in covid dataframe
covid['infectionRate'] = covid['positive'].div(covid['population'], fill_value=0)
covid['positiveRate'] = covid['positive'].div(covid['totalTestResults'], fill_value=0)
covid['negativeRate'] = covid['negative'].div(covid['totalTestResults'], fill_value=0)
covid['hospitalRate'] = covid['hospitalized'].div(covid['positive'], fill_value=0)
covid['deathRate'] = covid['death'].div(covid['positive'], fill_value=0)
covid['testRate'] = covid['totalTestResults'].div(covid['population'], fill_value=0)
covid['positivePC'] = covid['infectionRate'].mul(1000000)
covid['hospitalPC'] = covid['positivePC'].mul(covid['hospitalRate'])
covid['deathPC'] = covid['positivePC'].mul(covid['deathRate'])
covid['testPC'] = covid['testRate'].mul(1000000)

# Reorder dataframe
covid = covid[['state_name',
               'state',
               'population',
               'positive',
               'negative',
               'pending',
               'totalTestResults',
               'infectionRate',
               'positiveRate',
               'negativeRate',
               'hospitalized',
               'hospitalRate',
               'death',
               'deathRate',
               'testRate',
               'positivePC',
               'hospitalPC',
               'deathPC',
               'testPC',
             ]]

# Create index list for for bokeh graphs
index = covid['state_name'].to_list()

# Set new index for covid dataframe
covid.set_index('state_name', inplace=True)

# Sum US Statistics from covid dataframe 
total_population = int(covid['population'].sum())
total_positives = int(covid['positive'].sum())
total_negatives = int(covid['negative'].sum())
total_pendings = int(covid['pending'].sum())
total_hospitalized = int(covid['hospitalized'].sum())
total_test_results = int(covid['totalTestResults'].sum())
total_deaths = int(covid['death'].sum())

# Calculate rates for US Statistics
infection_rate = total_positives / total_population
positive_rate = total_positives / total_test_results
negative_rate = total_negatives / total_test_results
pending_rate = total_pendings / total_test_results
hospitalization_rate = total_hospitalized / total_positives
death_rate = total_deaths / total_positives
test_rate = total_test_results / total_population
positive_pc = infection_rate * 1000000
hospital_pc = positive_pc * hospitalization_rate
death_pc = positive_pc * death_rate
test_pc = test_rate * 1000000

# Create tracking data for various metrics to see how each state is performing
covid['expectedPositives'] = covid['population'].mul(infection_rate)
covid['positiveDelta'] = covid['positive'].sub(covid['expectedPositives'])
covid['positiveDeltaRate'] = covid['positiveDelta'].div(covid['expectedPositives'], fill_value=0)

covid['expectedNegatives'] = covid['totalTestResults'].mul(negative_rate)
covid['negativeDelta'] = covid['negative'].sub(covid['expectedNegatives'])
covid['negativeDeltaRate'] = covid['negativeDelta'].div(covid['expectedNegatives'], fill_value=0)

covid['expectedHospitalizations'] = covid['positive'].mul(hospitalization_rate)
covid['hospitalDelta'] = covid['hospitalized'].sub(covid['expectedHospitalizations'])
covid['hospitalDeltaRate'] = covid['hospitalDelta'].div(covid['expectedHospitalizations'], fill_value=0)

covid['expectedDeaths'] = covid['positive'].mul(death_rate)
covid['deathDelta'] = covid['death'].sub(covid['expectedDeaths'])
covid['deathDeltaRate'] = covid['deathDelta'].div(covid['expectedDeaths'], fill_value=0)

covid['expectedTests'] = covid['population'].mul(test_rate)
covid['testDelta'] = covid['totalTestResults'].sub(covid['expectedTests'])
covid['testDeltaRate'] = covid['testDelta'].div(covid['expectedTests'], fill_value=0)

covid['expectedPending'] = covid['totalTestResults'].mul(pending_rate)
covid['pendingDelta'] = covid['pending'].sub(covid['expectedPending'])
covid['pendingDeltaRate'] = covid['pendingDelta'].div(covid['expectedPending'], fill_value=0)

covid['populationShare'] = covid['population'].div(total_population, fill_value=0)
covid['positiveShare'] = covid['positive'].div(total_positives, fill_value=0)
covid['negativeShare'] = covid['negative'].div(total_negatives, fill_value=0)
covid['pendingShare'] = covid['pending'].div(total_pendings, fill_value=0)
covid['hospitalShare'] = covid['hospitalized'].div(total_hospitalized, fill_value=0)
covid['deathShare'] = covid['death'].div(total_deaths, fill_value=0)
covid['testShare'] = covid['totalTestResults'].div(total_test_results, fill_value=0)

Loading BokehJS ...

In [2]:
# Define which states to track in the markdown sections below. 
# Standard Capitalization for each state name in the comma-separated list
states_to_watch = ["Tennessee", "California", "Ohio", "Florida", "New York"]

## SARS-CoV-2 / COVID-19 Testing

In [3]:
stat_tuples_array = [('Positive Population %', "infectionRate", infection_rate, 0),
                     ('Tested Per 1M', "testPC", test_pc, 1),
                     ('Positive Per 1M', "positivePC", positive_pc, 1),
                     ('Positive Test Rate', "positiveRate", positive_rate, 0),
                     ('Negative Test Rate', "negativeRate", negative_rate, 0),
                     ('Expected Positives', "expectedPositives", '-----', 1),
                     ('Actual Positives', "positive", total_positives, 1),
                     ("Positive Delta", "positiveDelta", '-----', 1),
                     ('Delta %', "positiveDeltaRate", '-----', 0),
                     ('Expected Tests', "expectedTests", '-----', 1),
                     ('Actual Tests', "totalTestResults", total_test_results, 1),
                     ('Test Delta', "testDelta", '-----', 1),
                     ('Delta %', "testDeltaRate", '-----', 0),
                     ('Expected Negatives', "expectedNegatives", '-----', 1),
                     ('Actual Negatives', "negative", total_negatives, 1),
                     ('Negative Delta', "negativeDelta", '-----', 1),
                     ('Delta %', "negativeDeltaRate", '-----', 0),
                     ('Expected Pending', "expectedPending", '-----', 1),
                     ('Actual Pending', "pending", total_pendings, 1),
                     ('Pending Delta', "pendingDelta", '-----', 1),
                     ('Delta %', "pendingDeltaRate", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `California` | `Ohio` | `Florida` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: | ---: |
| Positive Population % | `0.0223%` | `0.0143%` | `0.0141%` | `0.0225%` | `0.3061%` | `0.0426%` |
| Tested Per 1M | `2,982` | `657` | `1,759` | `2,227` | `8,866` | `2,565` |
| Positive Per 1M | `222` | `142` | `140` | `225` | `3,061` | `425` |
| Positive Test Rate | `7.4706%` | `21.7390%` | `7.9990%` | `10.1025%` | `34.5283%` | `16.5897%` |
| Negative Test Rate | `92.5294%` | `78.2610%` | `92.0010%` | `89.8975%` | `65.4717%` | `83.4103%` |
| Expected Positives | `2,935` | `16,999` | `5,000` | `9,361` | `8,274` | `-----` |
| Actual Positives | `1,537` | `5,708` | `1,653` | `4,950` | `59,513` | `141,024` |
| Positive Delta | `-1,398` | `-11,291` | `-3,347` | `-4,411` | `51,238` | `-----` |
| Delta % | `-47.6483%` | `-66.4218%` | `-66.9422%` | `-47.1220%` | `619.2151%` | `-----` |
| Expected Tests | `17,697` | `102,468` | `30,141` | `56,427` | `49,878` | `-----` |
| Actual Tests | `20,574` | `26,257` | `20,665` | `48,998` | `172,360` | `850,072` |
| Test Delta | `2,876` | `-76,211` | `-9,476` | `-7,429` | `122,481` | `-----` |
| Delta % | `16.2554%` | `-74.3755%` | `-31.4395%` | `-13.1669%` | `245.5578%` | `-----` |
| Expected Negatives | `17,160` | `21,901` | `17,236` | `40,869` | `143,766` | `-----` |
| Actual Negatives | `19,037` | `20,549` | `19,012` | `44,048` | `112,847` | `709,048` |
| Negative Delta | `1,876` | `-1,352` | `1,775` | `3,178` | `-30,919` | `-----` |
| Delta % | `10.9328%` | `-6.1735%` | `10.2992%` | `7.7775%` | `-21.5065%` | `-----` |
| Expected Pending | `1,562` | `1,994` | `1,569` | `3,721` | `13,090` | `-----` |
| Actual Pending | `0` | `64,400` | `0` | `0` | `0` | `64,563` |
| Pending Delta | `-1,562` | `62,405` | `-1,569` | `-3,721` | `-13,090` | `-----` |
| Delta % | `-100.0000%` | `3129.3326%` | `-100.0000%` | `-100.0000%` | `-100.0000%` | `-----` |


## Hospitalizations

In [4]:
stat_tuples_array = [('Hospitalization %', "hospitalRate", hospitalization_rate, 0),
                     ('Positive Per 1M', "positivePC", positive_pc, 1),
                     ('Hospital Per 1M', "hospitalPC", hospital_pc, 1),
                     ('Actual Positives', "positive", total_positives, 1),
                     ('Expected Hospitalizations', "expectedHospitalizations", '-----', 1),
                     ('Actual Hospitalizations', "hospitalized", total_hospitalized, 1),
                     ('Hospitalized Delta', "hospitalDelta", '-----', 1),
                     ('Delta %', "hospitalDeltaRate", '-----', 0),
                     ('Hospital Share', "hospitalShare", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `California` | `Ohio` | `Florida` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: | ---: |
| Hospitalization % | `8.6532%` | `18.1149%` | `24.3799%` | `12.7879%` | `20.2897%` | `14.0558%` |
| Positive Per 1M | `222` | `142` | `140` | `225` | `3,061` | `425` |
| Hospital Per 1M | `19` | `25` | `34` | `28` | `621` | `59` |
| Actual Positives | `1,537` | `5,708` | `1,653` | `4,950` | `59,513` | `141,024` |
| Expected Hospitalizations | `216` | `802` | `232` | `695` | `8,365` | `-----` |
| Actual Hospitalizations | `133` | `1,034` | `403` | `633` | `12,075` | `19,822` |
| Hospitalized Delta | `-83` | `231` | `170` | `-62` | `3,709` | `-----` |
| Delta % | `-38.4365%` | `28.8790%` | `73.4514%` | `-9.0204%` | `44.3514%` | `-----` |
| Hospital Share | `0.6710%` | `5.2164%` | `2.0331%` | `3.1934%` | `60.9172%` | `-----` |


## Mortality

In [5]:
stat_tuples_array = [('Death Rate', "deathRate", death_rate, 0),
                     ('Positive Per 1M', "positivePC", positive_pc, 1),
                     ('Death Per 1M', "deathPC", death_pc, 1),
                     ('Actual Positives', "positive", total_positives, 1),
                     ('Actual Hospitalizations', "hospitalized", total_hospitalized, 1),
                     ('Hospitalization %', "hospitalRate", hospitalization_rate, 0),
                     ('Expected Deaths', "expectedDeaths", '-----', 1),
                     ('Actual Deaths', "death", total_deaths, 1),
                     ('Deaths Delta', "deathDelta", '-----', 1),
                     ('Delta %', "deathDeltaRate", '-----', 0),
                     ('Death Share', "deathShare", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `California` | `Ohio` | `Florida` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: | ---: |
| Death Rate | `0.4554%` | `2.1549%` | `1.7544%` | `1.1919%` | `1.6215%` | `1.7309%` |
| Positive Per 1M | `222` | `142` | `140` | `225` | `3,061` | `425` |
| Death Per 1M | `1` | `3` | `2` | `2` | `49` | `7` |
| Actual Positives | `1,537` | `5,708` | `1,653` | `4,950` | `59,513` | `141,024` |
| Actual Hospitalizations | `133` | `1,034` | `403` | `633` | `12,075` | `19,822` |
| Hospitalization % | `8.6532%` | `18.1149%` | `24.3799%` | `12.7879%` | `20.2897%` | `14.0558%` |
| Expected Deaths | `26` | `98` | `28` | `85` | `1,030` | `-----` |
| Actual Deaths | `7` | `123` | `29` | `59` | `965` | `2,441` |
| Deaths Delta | `-19` | `24` | `0` | `-26` | `-65` | `-----` |
| Delta % | `-73.6883%` | `24.4934%` | `1.3562%` | `-31.1392%` | `-6.3213%` | `-----` |
| Death Share | `0.2868%` | `5.0389%` | `1.1880%` | `2.4170%` | `39.5330%` | `-----` |


## Percent Share of Total US

In [6]:
stat_tuples_array = [('Population Share', "populationShare", '-----', 0),
                     ('Positive Share', "positiveShare", '-----', 0),
                     ('Hospital Share', "hospitalShare", '-----', 0),
                     ('Death Share', "deathShare", '-----', 0),
                     ('Test Share', "testShare", '-----', 0),
                     ('Negative Share', "negativeShare", '-----', 0),
                     ('Pending Share', "pendingShare", '-----', 0),
                    ]
table = " | Metric | "
for state in states_to_watch:
    table += "`{}` | ".format(state)
table += "`United States` |\n"
table += "| --- | "
for state in states_to_watch:
    table += "---: | "
table += "---: |\n"
for field in stat_tuples_array:
    table += "| {} | ".format(field[0])
    if field[3] == 0:
        for state in states_to_watch:
            table += "`{:.4%}` | ".format(covid.loc[state, field[1]])
        if isinstance(field[2], float):
            table += "`{:.4%}` |\n".format(field[2])
        else:
            table += "`{}` |\n".format(field[2])
    elif field[3] == 1:
        for state in states_to_watch:
            table += "`{:,}` | ".format(int(covid.loc[state, field[1]]))
        if isinstance(field[2], float) or isinstance(field[2], int):
            table += "`{:,}` |\n".format(int(field[2]))
        else:
            table += "`{}` |\n".format(field[2])

md(table)

 | Metric | `Tennessee` | `California` | `Ohio` | `Florida` | `New York` | `United States` |
| --- | ---: | ---: | ---: | ---: | ---: | ---: |
| Population Share | `2.0819%` | `12.0541%` | `3.5457%` | `6.6380%` | `5.8676%` | `-----` |
| Positive Share | `1.0899%` | `4.0475%` | `1.1721%` | `3.5100%` | `42.2006%` | `-----` |
| Hospital Share | `0.6710%` | `5.2164%` | `2.0331%` | `3.1934%` | `60.9172%` | `-----` |
| Death Share | `0.2868%` | `5.0389%` | `1.1880%` | `2.4170%` | `39.5330%` | `-----` |
| Test Share | `2.4203%` | `3.0888%` | `2.4310%` | `5.7640%` | `20.2759%` | `-----` |
| Negative Share | `2.6849%` | `2.8981%` | `2.6813%` | `6.2123%` | `15.9153%` | `-----` |
| Pending Share | `0.0000%` | `99.7475%` | `0.0000%` | `0.0000%` | `0.0000%` | `-----` |


## Charts

In [7]:
source = ColumnDataSource(data=covid)
plot_height = 1000
plot_width = 800
width = 0.5

panels = []

title_p = "COVID Confirmed: {:,} - Death Rate: {:.2%} - Hosp. Rate: {:.2%}".format(total_positives, death_rate, hospitalization_rate)
title_q = "COVID Deaths: {:,} - Death Rate: {:.2%}".format(total_deaths, death_rate)
title_z = "COVID Hospitalized: {:,} - Hospitalization Rate: {:.2%}".format(total_hospitalized, hospitalization_rate)
title_x = "COVID US National Infection Rate: {:.4%}".format(infection_rate)
title_y = "COVID US Confirmed / Expected Delta"
title_m = "COVID US Percentage Share of Confirmed Cases by State"
title_n = "COVID US Percentage Share of Deaths by State"
title_o = "COVID US Percentage Share of Hospitalizations by State" # FIX O

pTOOLTIPS = [
    ("State", "@state_name"),
    ("Cases", "@positive{0,0}"),
]
qTOOLTIPS = [
    ("State", "@state_name"),
    ("Deaths", "@death{0,0}"),
]
zTOOLTIPS = [
    ("State", "@state_name"),
    ("Hospitalized", "@hospitalized{0,0}"),
]
xTOOLTIPS = [
    ("State", "@state_name"),
    ("Inf. Rate", "@infectionRate{0.0000%}"),
]
yTOOLTIPS = [
    ("State", "@state_name"),
    ("Positive Delta", "@positiveDelta{0,0}"),
]
mTOOLTIPS = [
    ("State", "@state_name"),
    ("Positive Share", "@positiveShare{0.0000%}"),
]
nTOOLTIPS = [
    ("State", "@state_name"),
    ("Death Share", "@deathShare{0.0000%}"),
]
oTOOLTIPS = [
    ("State", "@state_name"),
    ("Hosp. Share", "@hospitalShare{0.0000%}"),
]

alpha_index = index[::-1]

p = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_p, toolbar_location="above", tooltips=pTOOLTIPS)
p.hbar(y='state_name', right='positive', height=width, source=source, line_color='white')
p.ygrid.grid_line_color = None
p.x_range.start = 0
p.x_range.end = covid['positive'].max() + covid['positive'].std()
#p.yaxis.major_label_orientation = math.pi / 4
p.xaxis.axis_label = "Confirmed Cases"
p.xaxis.formatter = NumeralTickFormatter(format="0,0")

q = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_q, toolbar_location="above", tooltips=qTOOLTIPS)
q.hbar(y='state_name', right='death', height=width, source=source, line_color='white')
q.ygrid.grid_line_color = None
q.x_range.start = 0
q.x_range.end = covid['death'].max() + covid['death'].std()
#q.xaxis.major_label_orientation = math.pi / 4
q.xaxis.axis_label = "Confirmed Deaths"
q.xaxis.formatter = NumeralTickFormatter(format="0,0")

z = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_z, toolbar_location="above", tooltips=zTOOLTIPS)
z.hbar(y='state_name', right='hospitalized', height=width, source=source, line_color='white')
z.ygrid.grid_line_color = None
z.x_range.start = 0
z.x_range.end = covid['hospitalized'].max() + covid['hospitalized'].std()
#z.xaxis.major_label_orientation = math.pi / 4
z.xaxis.axis_label = "Confirmed Hospitalizations"
z.xaxis.formatter = NumeralTickFormatter(format="0,0")

x = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_x, toolbar_location="above", tooltips=xTOOLTIPS)
x.hbar(y='state_name', right='infectionRate', height=width, source=source)
x.ygrid.grid_line_color = None
x.x_range.start = 0
x.x_range.end = covid['infectionRate'].max() + covid['infectionRate'].std()
#x.xaxis.major_label_orientation = math.pi / 4
x.xaxis.axis_label = "Confirmed Cases as % of State Population"
x.xaxis.formatter = NumeralTickFormatter(format="0.0000%")

y = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_y, toolbar_location="above", tooltips=yTOOLTIPS)
y.hbar(y='state_name', right='positiveDelta', height=width, source=source)
y.ygrid.grid_line_color = None
y.x_range.start = covid['positiveDelta'].min() - covid['positiveDelta'].std()
y.x_range.end = covid['positiveDelta'].max() + covid['positiveDelta'].std()
#y.xaxis.major_label_orientation = math.pi / 4
y.xaxis.axis_label = "Actual Covid Cases - Expected Covid Cases"
y.xaxis.formatter = NumeralTickFormatter(format="0,0")

m = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_m, toolbar_location="above", tooltips=mTOOLTIPS)
m.hbar(y='state_name', right='positiveShare', height=width, source=source)
m.ygrid.grid_line_color = None
m.x_range.start = 0
m.x_range.end = covid['positiveShare'].max() + covid['positiveShare'].std()
#m.xaxis.major_label_orientation = math.pi / 4
m.xaxis.axis_label = "Confirmed Cases as % of Total US Cases"
m.xaxis.formatter = NumeralTickFormatter(format="0.0000%")

n = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_n, toolbar_location="above", tooltips=nTOOLTIPS)
n.hbar(y='state_name', right='deathShare', height=width, source=source)
n.ygrid.grid_line_color = None
n.x_range.start = 0
n.x_range.end = covid['deathShare'].max() + covid['deathShare'].std()
#n.xaxis.major_label_orientation = math.pi / 4
n.xaxis.axis_label = "Confirmed Deaths as % of Total US Covid Deaths"
n.xaxis.formatter = NumeralTickFormatter(format="0.0000%")

o = figure(y_range=alpha_index, plot_height=plot_height, plot_width=plot_width, title=title_o, toolbar_location="above", tooltips=oTOOLTIPS)
o.hbar(y='state_name', right='hospitalShare', height=width, source=source)
o.ygrid.grid_line_color = None
o.x_range.start = 0
o.x_range.end = covid['hospitalShare'].max() + covid['hospitalShare'].std()
#o.xaxis.major_label_orientation = math.pi / 4
o.xaxis.axis_label = "Confirmed Recoveries as % of Total US Recoveries"
o.xaxis.formatter = NumeralTickFormatter(format="0.0000%")

charts = [
    (x, "Infection Rate"),
    (y, "Positive Delta"),
    (m, "Positive Share"),
    (n, "Death Share"),
    (p, "Positives"),
    (q, "Deaths"),
    (z, "Hospitalized"),
    (o, "Hospitalized Share"),
]

for viz, title in charts:
    panel = Panel(child=viz, title=title)
    panels.append(panel)

tabs = Tabs(tabs=panels)

show(tabs)